In [1]:
import re
import moval
import itertools
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

## Let's check the model parameters first (pickle)

In [12]:
# pickle_path_org = 'HAM_classification_auc_max_class_probability-conf_ts-model_False.pkl'
# pickle_path_cs = 'HAM_classification_auc_max_class_probability-conf_ts-model_True.pkl'
pickle_path_org = 'Cardiac_segmentation_f1score_max_class_probability-conf_atc-model_False.pkl'
pickle_path_cs = 'Cardiac_segmentation_f1score_max_class_probability-conf_atc-model_True.pkl'

In [13]:
with open(pickle_path_org, 'rb') as file:
    loaded_ckpt = pickle.load(file)
org_param = loaded_ckpt["param"]
print(org_param)

[0.98358307]


In [7]:
loaded_ckpt["fitted_perf"]

[array([1.74944455e-10, 9.55957348e-01, 9.22279793e-01, 9.37093276e-01]),
 array([1.74785451e-10, 9.63479688e-01, 9.20558498e-01, 9.49404762e-01]),
 array([1.72297938e-10, 9.51724138e-01, 8.81863561e-01, 8.60267316e-01]),
 array([1.70334537e-10, 9.26165804e-01, 8.38926176e-01, 8.50678736e-01]),
 array([1.68696650e-10, 9.00514581e-01, 7.78181820e-01, 2.89473778e-01]),
 array([1.67173760e-10, 7.75919736e-01, 3.97283542e-01, 1.00000000e+00]),
 array([1.66063303e-10, 4.01960814e-01, 2.37179512e-01, 1.00000000e+00]),
 array([1.67313613e-10, 9.20382167e-01, 8.51265825e-01, 8.41121500e-01]),
 array([1.69497271e-10, 9.52006295e-01, 8.83516485e-01, 9.06214690e-01]),
 array([1.71617841e-10, 9.55616439e-01, 8.98330805e-01, 9.28895613e-01]),
 array([1.73973556e-10, 9.74545455e-01, 9.20735574e-01, 9.45581628e-01]),
 array([1.75827267e-10, 9.71975878e-01, 9.17050692e-01, 9.44696970e-01]),
 array([1.77198143e-10, 9.64493222e-01, 8.77314816e-01, 9.27781393e-01]),
 array([1.79185780e-10, 9.57375038e-01

In [5]:
len(loaded_ckpt["fitted_perf"])

156

In [6]:
print(np.mean(loaded_ckpt["fitted_perf"]))

0.9688713121637205


In [7]:
with open(pickle_path_cs, 'rb') as file:
    loaded_ckpt = pickle.load(file)
cs_param = loaded_ckpt["param"]
print(cs_param)

[1.         4.59311228 5.41432676 6.112498  ]


In [8]:
loaded_ckpt["fitted_perf"]

[array([0.9997936 , 0.90921061, 0.85949425, 0.85081939]),
 array([0.99983804, 0.91945987, 0.86372489, 0.86736549]),
 array([0.99979262, 0.90819834, 0.82665436, 0.81944925]),
 array([0.99983078, 0.89640487, 0.78969442, 0.80531886]),
 array([0.99979363, 0.89150792, 0.76196788, 0.68613096]),
 array([0.99983904, 0.841678  , 0.69936717, 1.        ]),
 array([0.99982457, 0.75854206, 0.67855958, 1.        ]),
 array([0.99989676, 0.90627285, 0.80945939, 0.79029709]),
 array([0.99985138, 0.92200538, 0.83244782, 0.82204857]),
 array([0.99983685, 0.90313796, 0.84980394, 0.83661274]),
 array([0.99985678, 0.92908326, 0.87684925, 0.84385824]),
 array([0.99982735, 0.91268528, 0.8604991 , 0.81821451]),
 array([0.99977068, 0.89046712, 0.81563986, 0.78373519]),
 array([0.99972826, 0.8792495 , 0.83067513, 0.77812674]),
 array([0.99975138, 0.86992749, 0.83270498, 0.77454906]),
 array([0.99933107, 0.85183216, 0.78491083, 0.75250252]),
 array([0.999894  , 0.84621031, 0.75183686, 0.67002394]),
 array([0.9999

In [9]:
print(np.mean(loaded_ckpt["fitted_perf"]))

0.8759788441935924
